In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
from thesis_toolbox.plot.tools import map_large_scale
from dust.plot.maps import map_china
import copy
import pandas as pd

In [3]:
paths = snakemake.input


t_day = xr.open_dataset(paths[0]).resample(time='D').mean()
with xr.set_options(keep_attrs=True):
    mslp = xr.open_dataset(paths[1]).resample(time='D').mean()
    vort = xr.open_dataset(paths[2]).resample(time='D').mean()
    geopot = xr.open_dataset(paths[3]).resample(time='D').mean()
    mslp = mslp.assign(msl=mslp['msl']/100)
mslp['msl'].attrs['units'] = 'hPa'

geopot = geopot.where((mslp['msl']>1035) &(vort['vo'] < -1e-5), drop=True)

msl = mslp.sel(time=geopot.time)


In [5]:

filter_by_shi = True

std_clim_temp = t_day.resample(time='QS-Dec').std()
mean_clim_temp = t_day.resample(time='QS-Dec').mean()


In [6]:
winter_std = std_clim_temp.sel(time=(std_clim_temp.time.dt.month==12))
winter_std = winter_std.drop_sel(time=['1988-12-01T00:00:00.000000000','2019-12-01T00:00:00.000000000'])
winter_mean = mean_clim_temp.sel(time=(mean_clim_temp.time.dt.month==12))
winter_mean = winter_mean.drop_sel(time=['1988-12-01T00:00:00.000000000','2019-12-01T00:00:00.000000000'])

In [7]:
std = winter_std.mean(dim='time')


# t_day = mt2.resample(time='D').mean() 


t_drop =  t_day.shift(time=1) - t_day


t_anom = t_day- winter_mean.mean(dim='time')

t_anom = t_anom.drop_isel(time=0)

t_drop = t_drop.drop_isel(time=0)

t_drop_unfilterd = t_drop.copy()
if filter_by_shi:
    t_drop = t_drop.sel(time=msl.time)
    t_anom = t_anom.sel(time=msl.time)

t_day = t_day.sel(time=t_drop.time)

n_caobs = t_day.where((t_anom['t2m'] < -1.5*std),drop=True)

n_caobs = n_caobs.where((t_drop['t2m'] < -1.5*std), drop=True).time



In [8]:
t_caob = t_day.sel(time=n_caobs)

In [9]:
def get_events(n_caobs,t_drop, std):
    events = {}
    for t in n_caobs:
        str_t = str(t.dt.strftime('%Y-%m-%d').values)
    #     print(str_t)
        events[str_t] = {}

        caob_event = t_drop['t2m'].sel(time=slice(t,None))
        caob_event = iter(caob_event)
        temp_step = next(caob_event)
        time_list = []
        ndays=0
        while ((temp_step > 0.5*std['t2m']).mean(dim=['longitude','latitude']) < 0.20):
    #         print((temp_step > 0.5*std['t2m']).mean(dim=['longitude','latitude']))
    #         print((temp_step > -0.5*std).mean() < 0.75)
    #         print((temp_step > -0.5*std).mean(dim=['longitude','latitude']) < 0.75)
            time_list.append(temp_step.time)
            temp_step = next(caob_event)
            ndays += 1
        if not time_list:
            time_list = [temp_step.time]
        events[str_t]['time'] = xr.concat(time_list, dim='time')
        events[str_t]['ndays'] = ndays

    return events
    

In [10]:
events = get_events(n_caobs,t_drop_unfilterd,std)

In [11]:


k = list(events.keys())
clean_events = copy.deepcopy(events)
for i in range(1,len(k)):
    cur_event = events[k[i-1]]['time']
    next_event = events[k[i]]['time']
    if len(np.intersect1d(cur_event, next_event)) == len(next_event):
        clean_events.pop(k[i])
        

data_dict = {}
for event in clean_events:
    data_dict[event]={}
    data_dict[event]['sdate'] = clean_events[event]['time'][0].dt.date.values
    data_dict[event]['edate'] = clean_events[event]['time'][-1].dt.date.values
    data_dict[event]['ndays'] = clean_events[event]['ndays']
    
    

df = pd.DataFrame(data_dict).transpose()

df.index = pd.to_datetime(df.index)


df['intensity'] = [t_drop['t2m'].sel(time=slice(d[1]['sdate'],d[1]['edate'])).mean().values for d in df.iterrows()]

df = df[df['intensity'] < 0]

events_per_winter = df['sdate'].resample('QS-Dec').count()
mean_intensity = df['intensity'].resample('QS-Dec').mean()
total_duration = df['ndays'].resample('QS-Dec').sum()

winter_CAOBs_table =  pd.concat([events_per_winter,mean_intensity,total_duration],axis=1)
winter_CAOBs_table = winter_CAOBs_table.rename(columns={'sdate':'n_events', 'ndays':'n_CAOB_days'})
winter_CAOBs_table = winter_CAOBs_table[winter_CAOBs_table.index.month==12]
winter_CAOBs_table.index = winter_CAOBs_table.index.year
winter_CAOBs_table.to_csv(snakemake.output.csv)


In [12]:
fig, ax = plt.subplot_mosaic(
    """
    ab
    cc
    """,gridspec_kw={"height_ratios": [3, 1],"wspace": 0.06, "hspace":0.3}, figsize=(12,6))
winter_CAOBs_table['n_events'].plot.bar(ax=ax['a'], color='k')
ax['a'].set_ylabel('n CAOBs')
winter_CAOBs_table['intensity'].plot.bar(ax=ax['b'], color='k')

ax['b'].invert_yaxis()
ax['b'].yaxis.tick_right()
ax['b'].set_ylabel('Mean daily temperature drop [K]')
ax['b'].yaxis.set_label_position('right')
for l,ax_i in ax.items():
    ax_i.text(0.02,0.87,f'{l})',transform=ax_i.transAxes, fontsize=14)

winter_CAOBs_table['n_CAOB_days'].plot.bar(ax=ax['c'], color='k')
plt.savefig(snakemake.output.outpath, dpi=144, bbox_inches='tight')